In [1]:
import scGCO
import time
import pandas as pd
import numpy as np
# %load_ext memory_profiler

In [2]:
nc=[]
tt =[]
import time

In [3]:

n_cell=[100,250,500,1000,1500,5000,10000,15000]
for i in n_cell:
    ff='Data/1_million_cells_100genes_counts_0.csv'
    data0 = pd.read_csv(ff, sep=',', index_col = 0,iterator=True)
    data=data0.get_chunk(i)
    data=pd.read_csv(ff,sep=',',index_col=0)
    
    temp = [val.split('x') for val in data.index.values]
    coord = np.array([[float(a[0]), float(a[1])] for a in temp])

    locs=coord
    sub_data=data
    sub_data = sub_data.loc[:,(sub_data != 0).astype(int).sum(axis=0) >= 10]    
    data_norm = scGCO.normalize_count_cellranger(sub_data)

    factor_df, size_factor = scGCO.estimate_smooth_factor(locs, data_norm)

    start = time.time()
    result_df = scGCO.identify_spatial_genes(locs, data_norm, size_factor)
    end = time.time()
    t=end-start
    
    nc.append(int(i))
    tt.append(t)
     


100%|██████████| 8/8 [3:11:33<00:00, 879.90s/it]    


FileNotFoundError: File b'Data/1_million_cells_100genes_counts_0.csv' does not exist

In [ ]:
nc

In [ ]:
memory=np.array([145.54 ,152.25,159.99,167.42,190.46,245.66,358.39, 675.57,1216.41,2094.42 ])

In [ ]:
df=pd.DataFrame({'n.cells':nc,'n.genes':100,'time_s':tt})

In [ ]:
df.to_excel('simulate/Result/scGCO_100gene.xls')

In [ ]:
df